# Exercise 6

## SVM & Regularization


For this homework we consider a set of observations on a number of red and white wine varieties involving their chemical properties and ranking by tasters. Wine industry shows a recent growth spurt as social drinking is on the rise. The price of wine depends on a rather abstract concept of wine appreciation by wine tasters, opinion among whom may have a high degree of variability. Pricing of wine depends on such a volatile factor to some extent. Another key factor in wine certification and quality assessment is physicochemical tests which are laboratory-based and takes into account factors like acidity, pH level, presence of sugar and other chemical properties. For the wine market, it would be of interest if human quality of tasting can be related to the chemical properties of wine so that certification and quality assessment and assurance process is more controlled.

Two datasets are available of which one dataset is on red wine and have 1599 different varieties and the other is on white wine and have 4898 varieties. All wines are produced in a particular area of Portugal. Data are collected on 12 different properties of the wines one of which is Quality, based on sensory data, and the rest are on chemical properties of the wines including density, acidity, alcohol content etc. All chemical properties of wines are continuous variables. Quality is an ordinal variable with possible ranking from 1 (worst) to 10 (best). Each variety of wine is tasted by three independent tasters and the final rank assigned is the median rank given by the tasters.

A predictive model developed on this data is expected to provide guidance to vineyards regarding quality and price expected on their produce without heavy reliance on volatility of wine tasters.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [3]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
5991,9.2,0.310,0.36,2.2,0.079,11.0,31.0,0.99615,3.33,0.86,12.0,7,red
4201,6.7,0.410,0.27,2.6,0.033,25.0,85.0,0.99086,3.05,0.34,11.7,6,white
4928,6.7,0.675,0.07,2.4,0.089,17.0,82.0,0.99580,3.35,0.54,10.1,5,red
1863,6.6,0.280,0.30,7.8,0.049,57.0,202.0,0.99580,3.24,0.39,9.5,5,white
3457,6.1,0.105,0.31,1.3,0.037,55.0,145.0,0.99120,3.41,0.41,11.1,7,white


# Exercise 6.1

Show the frecuency table of the quality by type of wine

In [11]:
import warnings
warnings.filterwarnings('ignore')

#Red Wine #

freq_tab_r = pd.value_counts(data_r["quality"]).sort_index()
print("red wine \n",freq_tab_r)

# White Wine #
freq_tab_w = pd.value_counts(data_w["quality"]).sort_index()
print("white wine \n",freq_tab_w)

# Red and white
freq_tab = pd.value_counts(data["quality"]).sort_index()
print("Red and white\n",freq_tab)



red wine 
 3     10
4     53
5    681
6    638
7    199
8     18
Name: quality, dtype: int64
white wine 
 3      20
4     163
5    1457
6    2198
7     880
8     175
9       5
Name: quality, dtype: int64
Red and white
 3      30
4     216
5    2138
6    2836
7    1079
8     193
9       5
Name: quality, dtype: int64


Hay 7 categorias en total que clasifica el vino de acuerdo a la calidad. 

# SVM

# Exercise 6.2

* Standarized the features (not the quality)
* Create a binary target for each type of wine
* Create two Linear SVM's for the white and red wines, repectively.


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC # "Support Vector Classifier"
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, f1_score, accuracy_score, confusion_matrix, log_loss

In [15]:
#Punto b 
data_r = data_r.assign(type=1)
data_w = data_w.assign(type=0)
data_r.head()
data_w.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0


In [19]:
# a
# Red Wine
x1_1 = data_r['fixed acidity'].values.reshape(-1, 1)
x1_2 = data_r['volatile acidity'].values.reshape(-1, 1)
x1_3 = data_r['citric acid'].values.reshape(-1, 1)
x1_4 = data_r['residual sugar'].values.reshape(-1, 1)
x1_5 = data_r['chlorides'].values.reshape(-1, 1)
x1_6 = data_r['free sulfur dioxide'].values.reshape(-1, 1)
x1_7 = data_r['total sulfur dioxide'].values.reshape(-1, 1)
x1_8 = data_r['density'].values.reshape(-1, 1)
x1_9 = data_r['pH'].values.reshape(-1, 1)
x1_10 = data_r['sulphates'].values.reshape(-1, 1)
x1_11 = data_r['alcohol'].values.reshape(-1, 1)
x1_12 = data_r['type'].values.reshape(-1, 1)
X1 = np.c_[x1_1, x1_2, x1_3, x1_4, x1_5, x1_6, x1_7, x1_8, x1_9, x1_10, x1_11, x1_12]

ss1 = StandardScaler(with_mean=True, with_std=True)
ss1.fit(X1.astype(np.float))
X1_n = ss1.transform(X1.astype(np.float))

Y1 = data_r["quality"]

# White Wine
x2_1 = data_w['fixed acidity'].values.reshape(-1, 1)
x2_2 = data_w['volatile acidity'].values.reshape(-1, 1)
x2_3 = data_w['citric acid'].values.reshape(-1, 1)
x2_4 = data_w['residual sugar'].values.reshape(-1, 1)
x2_5 = data_w['chlorides'].values.reshape(-1, 1)
x2_6 = data_w['free sulfur dioxide'].values.reshape(-1, 1)
x2_7 = data_w['total sulfur dioxide'].values.reshape(-1, 1)
x2_8 = data_w['density'].values.reshape(-1, 1)
x2_9 = data_w['pH'].values.reshape(-1, 1)
x2_10 = data_w['sulphates'].values.reshape(-1, 1)
x2_11 = data_w['alcohol'].values.reshape(-1, 1)
x2_12 = data_w['type'].values.reshape(-1, 1)
X2 = np.c_[x2_1, x2_2, x2_3, x2_4, x2_5, x2_6, x2_7, x2_8, x2_9, x2_10, x2_11, x2_12]

ss2 = StandardScaler(with_mean=True, with_std=True)
ss2.fit(X2.astype(np.float))
X2_n = ss2.transform(X2.astype(np.float))

Y2 = data_w["quality"]


In [25]:
# c
# Red Wine
#   a. linear
clf1 = SVC(kernel='linear')
clf1.fit(X1_n, Y1) #Ajustar el modelo

Y1_predictions = clf1.predict(X1_n)
print('\nRed_linear\n')
print("Accuracy:", accuracy_score(Y1, Y1_predictions))  
print('Confusion Matrix: \n', confusion_matrix(Y1, Y1_predictions))

# White Wine
#   a. linear
clf2 = SVC(kernel='linear')
clf2.fit(X2_n, Y2)

Y2_predictions = clf2.predict(X2_n)

print('\nWhite_linear\n')
print("Accuracy:", accuracy_score(Y2, Y2_predictions))  
print('Confusion Matrix: \n', confusion_matrix(Y2, Y2_predictions))




Red_linear

Accuracy: 0.5866166353971232
Confusion Matrix: 
 [[  0   0   9   1   0   0]
 [  0   0  39  14   0   0]
 [  0   0 529 152   0   0]
 [  0   0 229 409   0   0]
 [  0   0  13 186   0   0]
 [  0   0   0  18   0   0]]

White_linear

Accuracy: 0.5216414863209473
Confusion Matrix: 
 [[   0    0    8   12    0    0    0]
 [   0    0  102   61    0    0    0]
 [   0    0  780  677    0    0    0]
 [   0    0  423 1775    0    0    0]
 [   0    0   43  837    0    0    0]
 [   0    0    1  174    0    0    0]
 [   0    0    0    5    0    0    0]]


**RTA/**La maquina de soporte vectorial se ajusto bien a esos datos. El 58% de los datos del vino rojo se ajustaron correctamente, mientras que con el vino blanco se ajustaron solo el 52%. 

# Exercise 6.3

Test the two SVM's using the different kernels (‘poly’, ‘rbf’, ‘sigmoid’)


In [30]:
clf3a = SVC(kernel='poly') #polinomico
clf3a.fit(X1_n, Y1) #Ajustar el modelo
Y3a_predictions = clf3a.predict(X1_n)
print('\nRed_poly\n')
print("Accuracy:", accuracy_score(Y1, Y3a_predictions))  
print('Confusion Matrix: \n', confusion_matrix(Y1, Y3a_predictions))

clf3b = SVC(kernel='poly')
clf3b.fit(X2_n, Y2)
Y3b_predictions = clf3b.predict(X2_n)
print('\nwhite_poly\n')
print("Accuracy:", accuracy_score(Y2, Y3b_predictions))  
print('Confusion Matrix: \n', confusion_matrix(Y2, Y3b_predictions))


Red_poly

Accuracy: 0.6760475297060663
Confusion Matrix: 
 [[  3   0   5   2   0   0]
 [  0   7  36   9   1   0]
 [  0   0 575 105   1   0]
 [  0   0 211 419   8   0]
 [  0   0  15 109  75   0]
 [  0   0   0  12   4   2]]

white_poly

Accuracy: 0.5681910984075133
Confusion Matrix: 
 [[  11    0    3    6    0    0    0]
 [   0   36   58   69    0    0    0]
 [   0    5  611  838    3    0    0]
 [   0    1  184 1971   42    0    0]
 [   0    0   15  717  146    2    0]
 [   0    0    0  131   36    8    0]
 [   0    0    0    3    2    0    0]]


**RTA/**Cuando corremos el modelo mediante una regresion polinomica, encontramos para el vino rojo un ajuste del 67.60% y del vino blanco 56,81%. 

In [33]:
clf3c = SVC(kernel='rbf')#exponencial
clf3c.fit(X1_n, Y1) #Ajustar el modelo
Y3c_predictions = clf3c.predict(X1_n)
print('\nRed_poly\n')
print("Accuracy:", accuracy_score(Y1, Y3c_predictions))  
print('Confusion Matrix: \n', confusion_matrix(Y1, Y3c_predictions))

clf3d = SVC(kernel='rbf')
clf3d.fit(X2_n, Y2)
Y3d_predictions = clf3d.predict(X2_n)
print('\nwhite_poly\n')
print("Accuracy:", accuracy_score(Y2, Y3d_predictions))  
print('Confusion Matrix: \n', confusion_matrix(Y2, Y3d_predictions))


Red_poly

Accuracy: 0.66729205753596
Confusion Matrix: 
 [[  0   0   7   3   0   0]
 [  0   1  38  13   1   0]
 [  0   0 545 134   2   0]
 [  0   0 171 450  17   0]
 [  0   0  12 116  71   0]
 [  0   0   0  12   6   0]]

white_poly

Accuracy: 0.6108615761535321
Confusion Matrix: 
 [[   1    0    8   11    0    0    0]
 [   0   17   98   46    2    0    0]
 [   0    1  927  525    4    0    0]
 [   0    0  344 1778   76    0    0]
 [   0    0   28  583  269    0    0]
 [   0    0    1  127   47    0    0]
 [   0    0    0    3    2    0    0]]


**RTA/**Cuando corremos el modelo mediante una regresion exponencial, encontramos para el vino rojo un ajuste del 66.72% y del vino blanco 61,08%. 

In [34]:
clf3e = SVC(kernel='sigmoid') #0 
clf3e.fit(X1_n, Y1) #Ajustar el modelo
Y3e_predictions = clf3e.predict(X1_n)
print('\nRed_poly\n')
print("Accuracy:", accuracy_score(Y1, Y3e_predictions))  
print('Confusion Matrix: \n', confusion_matrix(Y1, Y3e_predictions))

clf3f = SVC(kernel='sigmoid')
clf3f.fit(X2_n, Y2)
Y3f_predictions = clf3f.predict(X2_n)
print('\nwhite_poly\n')
print("Accuracy:", accuracy_score(Y2, Y3f_predictions))  
print('Confusion Matrix: \n', confusion_matrix(Y2, Y3f_predictions))


Red_poly

Accuracy: 0.49468417761100686
Confusion Matrix: 
 [[  0   0   3   7   0   0]
 [  0   0  27  22   3   1]
 [  0   5 408 230  38   0]
 [  0   2 215 341  80   0]
 [  0   0  34 123  42   0]
 [  0   0   2   8   8   0]]

white_poly

Accuracy: 0.42343813801551655
Confusion Matrix: 
 [[   0    2    9    6    3    0    0]
 [   1   15   73   66    6    2    0]
 [   0  104  635  647   68    3    0]
 [   1   71  599 1163  352   12    0]
 [   0   11  100  497  257   15    0]
 [   0    3    9  105   54    4    0]
 [   0    0    1    0    4    0    0]]


**RTA/**Cuando corremos el modelo mediante una regresion sigmoid, encontramos para el vino rojo un ajuste del 49.46% y del vino blanco 42.34%. 

# Exercise 6.4
Using the best SVM find the parameters that gives the best performance

'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]

In [35]:
C = [0.1, 1, 10, 100, 1000] #parametros de ajustes
gamma = [0.01, 0.001, 0.0001]

r_acc_scores = []
w_acc_scores = []

In [38]:
for i in C:
    for j in gamma:
        # Red Wine
        clf_4_r = SVC(C=i, gamma=j, kernel='poly')
        clf_4_r.fit(X1_n, Y1)
        Y_4_r_predictions = clf_4_r.predict(X1_n)
        r_acc_val = accuracy_score(Y1, Y_4_r_predictions)
        r_acc_scores.append(r_acc_val) #en las lineas pasadas se genera el valor y en este se almacena
        print(i)
        print(j)
        print(r_acc_val)

0.1
0.01
0.425891181988743
0.1
0.001
0.425891181988743
0.1
0.0001
0.425891181988743
1
0.01
0.42964352720450283
1
0.001
0.425891181988743
1
0.0001
0.425891181988743
10
0.01
0.4546591619762351
10
0.001
0.425891181988743
10
0.0001
0.425891181988743
100
0.01
0.5872420262664165
100
0.001
0.425891181988743
100
0.0001
0.425891181988743
1000
0.01
0.6891807379612258
1000
0.001
0.42964352720450283
1000
0.0001
0.425891181988743


**RTA/**
Para el vino blanco utilizamos la regresion pilinomica, revisando los resultados de cada combinacion posible, modificando el C y el gamma el mejor ajuste se da cuando c=100 y gamma=0.01, arrojando un ajuste del 68,91%. 

In [39]:
 for i in C:
    for j in gamma:
    # White Wine
        clf_4_w = SVC(C=i, gamma=j, kernel='rbf')
        clf_4_w.fit(X2_n, Y2)
        Y_4_w_predictions = clf_4_w.predict(X2_n)
        w_acc_val = accuracy_score(Y2, Y_4_w_predictions)
        w_acc_scores.append(w_acc_val)
        print(i)
        print(j)
        print(w_acc_val)

0.1
0.01
0.504695794201715
0.1
0.001
0.44875459371171905
0.1
0.0001
0.44875459371171905
1
0.01
0.5371580236831359
1
0.001
0.5048999591670069
1
0.0001
0.44875459371171905
10
0.01
0.5673744385463454
10
0.001
0.5244997958350347
10
0.0001
0.504695794201715
100
0.01
0.5998366680277665
100
0.001
0.5398121682319315
100
0.0001
0.5210289914250714
1000
0.01
0.6412821559820335
1000
0.001
0.5653327888934259
1000
0.0001
0.5267456104532462


**RTA/**
Para el vino blanco utilizamos la distribución rbf. Revisando los resultados de cada combinacion posible, modificando el C y el gamma el mejor ajuste se da cuando c=1000 y gamma=0.01, arrojando un ajuste del 64,12%. 

# Exercise 6.5

Compare the results with other methods

In [46]:
# Logistic Regresion vino rojo
from sklearn.linear_model import LogisticRegression

logreg5_a = LogisticRegression(C=1e9, solver='liblinear', multi_class='ovr')
logreg5_a.fit(X1_n, Y1)
print(logreg5_a.coef_)
Y_5_a_predictions = logreg5_a.predict(X1_n)
print('Accuracy: \n', accuracy_score(Y1, Y_5_a_predictions))
print('Confusion Matrix: \n', confusion_matrix(Y1, Y_5_a_predictions))
print('f1_Score: \n', f1_score(Y1, Y_5_a_predictions, average='micro'))

[[ 0.50110602  1.5049741   0.7218695  -0.20022028  0.61319872  1.60677548
  -3.26355301  0.88763353  1.23670196 -0.45652364 -1.40947889  0.        ]
 [ 0.62508096  0.62245144  0.09024381  0.46828107  0.16256151 -0.32138717
  -0.26571482 -0.78518054  0.68834373 -0.10778296 -0.68303345  0.        ]
 [-0.46245766  0.29285521  0.12813349 -0.19594133  0.12911284 -0.18438235
   0.58956891  0.36739883 -0.14721951 -0.45584256 -0.76676644  0.        ]
 [-0.09874784 -0.33423978 -0.26793647 -0.12015716 -0.0210899   0.25553592
  -0.4179403   0.28139118 -0.06506546  0.12143296  0.36897705  0.        ]
 [ 0.55136886 -0.5284807   0.04120626  0.33509974 -0.3234244   0.09979199
  -0.48888579 -0.47714007  0.12673663  0.57517202  0.68791151  0.        ]
 [-0.82985192  0.32133333  0.6351676  -0.12098712 -1.42773986  0.13651082
  -0.83947034  0.01842208 -1.03290439  0.63691262  1.28180236  0.        ]]
Accuracy: 
 0.6047529706066291
Confusion Matrix: 
 [[  1   1   7   1   0   0]
 [  0   1  34  16   2   0]


**RTA/**Corro una regresión logistica para cada uno de los vino, pára el vino Rojo en Accuracy no mejora, seguimos obteniendo un mejor resultado en Poly. 

In [49]:
# Logistic Regresion vino blanco
from sklearn.linear_model import LogisticRegression

logreg5_b = LogisticRegression(C=1e9, solver='liblinear', multi_class='ovr')
logreg5_b.fit(X2_n, Y2)
print(logreg5_a.coef_)
Y_5_b_predictions = logreg5_b.predict(X2_n)
print('Accuracy: \n', accuracy_score(Y2, Y_5_b_predictions))
print('Confusion Matrix: \n', confusion_matrix(Y2, Y_5_b_predictions))
print('f1_Score: \n', f1_score(Y2, Y_5_b_predictions, average='micro'))

[[ 0.50110602  1.5049741   0.7218695  -0.20022028  0.61319872  1.60677548
  -3.26355301  0.88763353  1.23670196 -0.45652364 -1.40947889  0.        ]
 [ 0.62508096  0.62245144  0.09024381  0.46828107  0.16256151 -0.32138717
  -0.26571482 -0.78518054  0.68834373 -0.10778296 -0.68303345  0.        ]
 [-0.46245766  0.29285521  0.12813349 -0.19594133  0.12911284 -0.18438235
   0.58956891  0.36739883 -0.14721951 -0.45584256 -0.76676644  0.        ]
 [-0.09874784 -0.33423978 -0.26793647 -0.12015716 -0.0210899   0.25553592
  -0.4179403   0.28139118 -0.06506546  0.12143296  0.36897705  0.        ]
 [ 0.55136886 -0.5284807   0.04120626  0.33509974 -0.3234244   0.09979199
  -0.48888579 -0.47714007  0.12673663  0.57517202  0.68791151  0.        ]
 [-0.82985192  0.32133333  0.6351676  -0.12098712 -1.42773986  0.13651082
  -0.83947034  0.01842208 -1.03290439  0.63691262  1.28180236  0.        ]]
Accuracy: 
 0.5377705185790118
Confusion Matrix: 
 [[   1    0    8   11    0    0    0]
 [   0    2   95

**RTA/**Para el vino Blanco en Accuracy no mejora, seguimos obteniendo un mejor resultado en rbf.  

# Regularization

# Exercise 6.6


* Train a linear regression to predict wine quality (Continous)

* Analyze the coefficients

* Evaluate the RMSE

In [50]:
data["type"] = data["type"].map({"white": 0, "red": 1})

x6_1 = data['fixed acidity'].values.reshape(-1, 1)
x6_2 = data['volatile acidity'].values.reshape(-1, 1)
x6_3 = data['citric acid'].values.reshape(-1, 1)
x6_4 = data['residual sugar'].values.reshape(-1, 1)
x6_5 = data['chlorides'].values.reshape(-1, 1)
x6_6 = data['free sulfur dioxide'].values.reshape(-1, 1)
x6_7 = data['total sulfur dioxide'].values.reshape(-1, 1)
x6_8 = data['density'].values.reshape(-1, 1)
x6_9 = data['pH'].values.reshape(-1, 1)
x6_10 = data['sulphates'].values.reshape(-1, 1)
x6_11 = data['alcohol'].values.reshape(-1, 1)
x6_12 = data['type'].values.reshape(-1, 1)
X6 = np.c_[x6_1, x6_2, x6_3, x6_4, x6_5, x6_6, x6_7, x6_8, x6_9, x6_10, x6_11, x6_12]

ss6 = StandardScaler(with_mean=True, with_std=True)
ss6.fit(X6.astype(np.float))
X6_n = ss6.transform(X6.astype(np.float))

Y6 = data["quality"]


In [51]:
# build a linear regression model
from sklearn.linear_model import LinearRegression
linreg6 = LinearRegression()
linreg6.fit(X6_n, Y6)

Y_6_predictions = linreg6.predict(X6_n)

print('\nLinreg\n')

print('coef_: \n', linreg6.coef_)
print('intercept_: \n', linreg6.intercept_)

print("RMSE: %.4f" % np.sqrt(mean_squared_error(Y6, Y_6_predictions)))
print('r2 score: %.4f' % r2_score(Y6, Y_6_predictions)) # 1 - fit, 0 - no fit



Linreg

coef_: 
 [ 0.11027401 -0.24568548 -0.00909927  0.29704168 -0.02652718  0.08762284
 -0.07927578 -0.311567    0.08018737  0.10739154  0.26556038  0.155642  ]
intercept_: 
 5.81837771279107
RMSE: 0.7324
r2 score: 0.2965


**RTA/**Erroe medio cuadratico, entre mas cercano a 0 es porque mas se ajusta el modelo. 
    r2 score: entre mas cercano a 1, mas se ajusta el modelo 
Con lo anterio, el modelo no se ajusta con una diferencia muy grande. 
    El que beta  mas grande es el q mas aporta al momento de definir la calidad de vino (Positivamnete) en este caso es residual sugar y negativamente que es la densidad. 

# Exercise 6.7

* Estimate a ridge regression with alpha equals 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [52]:
from sklearn.linear_model import Ridge # alpha=0 is equivalent to linear regression

# alpha = 0.1
print('\nRidgereg - 0.1\n')
ridgereg1 = Ridge(alpha=0.1)  # , normalize=True
ridgereg1.fit(X6_n, Y6)

Y_7_1_predictions = ridgereg1.predict(X6_n)

print('coef_: \n', ridgereg1.coef_)
print('intercept_: \n', ridgereg1.intercept_)

print("RMSE: %.4f" % np.sqrt(mean_squared_error(Y6, Y_7_1_predictions)))
print('r2 score: %.4f' % r2_score(Y6, Y_7_1_predictions))

# alpha = 1
print('\nRidgereg - 1\n')
ridgereg2 = Ridge(alpha=1)  # , normalize=True
ridgereg2.fit(X6_n, Y6)

Y_7_2_predictions = ridgereg2.predict(X6_n)

print('coef_: \n', ridgereg2.coef_)
print('intercept_: \n', ridgereg2.intercept_)

print("RMSE: %.4f" % np.sqrt(mean_squared_error(Y6, Y_7_2_predictions)))
print('r2 score: %.4f' % r2_score(Y6, Y_7_2_predictions))



Ridgereg - 0.1

coef_: 
 [ 0.11021354 -0.24567985 -0.00909761  0.29693471 -0.02653296  0.08762743
 -0.07928723 -0.31140594  0.08015225  0.10738163  0.26562142  0.15558096]
intercept_: 
 5.81837771279107
RMSE: 0.7324
r2 score: 0.2965

Ridgereg - 1

coef_: 
 [ 0.10967271 -0.24562903 -0.00908255  0.29597774 -0.02658472  0.08766835
 -0.07938954 -0.30996565  0.07983816  0.10729294  0.26616683  0.15503466]
intercept_: 
 5.818377712791067
RMSE: 0.7324
r2 score: 0.2965


# Exercise 6.8

* Estimate a lasso regression with alpha equals 0.01, 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [53]:
from sklearn.linear_model import Lasso # alpha=0 is equivalent to linear regression

# alpha = 0.01
print('\nLassoreg - 0.01\n')
lassoreg1 = Lasso(alpha=0.01)  # , normalize=True
lassoreg1.fit(X6_n, Y6)

Y_8_1_predictions = lassoreg1.predict(X6_n)

print('coef_: \n', lassoreg1.coef_)
print('intercept_: \n', lassoreg1.intercept_)

print("RMSE: %.4f" % np.sqrt(mean_squared_error(Y6, Y_8_1_predictions)))
print('r2 score: %.4f' % r2_score(Y6, Y_8_1_predictions))

# alpha = 0.1
print('\nLassoreg - 0.1\n')
lassoreg2 = Lasso(alpha=0.1)  # , normalize=True
lassoreg2.fit(X6_n, Y6)

Y_8_2_predictions = lassoreg2.predict(X6_n)

print('coef_: \n', lassoreg2.coef_)
print('intercept_: \n', lassoreg2.intercept_)

print("RMSE: %.4f" % np.sqrt(mean_squared_error(Y6, Y_8_2_predictions)))
print('r2 score: %.4f' % r2_score(Y6, Y_8_2_predictions))

# alpha = 1
print('\nLassoreg - 1\n')
lassoreg3 = Lasso(alpha=1)  # , normalize=True
lassoreg3.fit(X6_n, Y6)

Y_8_3_predictions = lassoreg3.predict(X6_n)

print('coef_: \n', lassoreg3.coef_)
print('intercept_: \n', lassoreg3.intercept_)

print("RMSE: %.4f" % np.sqrt(mean_squared_error(Y6, Y_8_3_predictions)))
print('r2 score: %.4f' % r2_score(Y6, Y_8_3_predictions))



Lassoreg - 0.01

coef_: 
 [-0.         -0.2266899  -0.          0.08427125 -0.02128916  0.07237462
 -0.07290899 -0.          0.01170913  0.07755272  0.38765188  0.02836949]
intercept_: 
 5.818377712790538
RMSE: 0.7366
r2 score: 0.2883

Lassoreg - 0.1

coef_: 
 [-0.         -0.12133808  0.          0.         -0.          0.
 -0.         -0.          0.          0.          0.28340642 -0.        ]
intercept_: 
 5.818377712790532
RMSE: 0.7641
r2 score: 0.2342

Lassoreg - 1

coef_: 
 [-0. -0.  0. -0. -0.  0. -0. -0.  0.  0.  0. -0.]
intercept_: 
 5.818377712790519
RMSE: 0.8732
r2 score: 0.0000


**RTA/** En este modelo de clasificacion los betas los va eliminado si no son gratificantes para el modelo. Como se observa cuando el alpha es 1 los betas se eliminan, el RMSE con mayor ajuste al modelo es alpha 0.01. 

# Exercise 6.9

* Create a binary target

* Train a logistic regression to predict wine quality (binary)

* Analyze the coefficients

* Evaluate the f1score

In [55]:
Y7 = data["quality"].map({3: 0, 4: 0, 5: 0, 6: 0, 7: 1, 8: 1, 9: 1})

logreg9 = LogisticRegression(C=1e9,  solver='liblinear')
logreg9.fit(X6_n, Y7)
print(logreg9.coef_)

Y_9_predictions = logreg9.predict(X6_n)
print('Histograma: \n', pd.value_counts(Y7).sort_index())
print('Accuracy: \n', accuracy_score(Y7, Y_9_predictions))
print('Confusion Matrix: \n', confusion_matrix(Y7, Y_9_predictions))
print('f1_Score: \n', f1_score(Y7, Y_9_predictions)) #  , average='micro' - Calculate metrics globally by counting the total true positives, false negatives and false positives.


[[ 0.64040034 -0.60303914 -0.03615616  1.04543735 -0.26664871  0.19165801
  -0.20940333 -1.27058901  0.4174233   0.36582077  0.54161572  0.33368393]]
Histograma: 
 0    5220
1    1277
Name: quality, dtype: int64
Accuracy: 
 0.8182237955979683
Confusion Matrix: 
 [[4974  246]
 [ 935  342]]
f1_Score: 
 0.36675603217158176


**RTA**El modelo tiene buena prediccion, el f1score por mattiz confusion

# Exercise 6.10

* Estimate a regularized logistic regression using:
* C = 0.01, 0.1 & 1.0
* penalty = ['l1, 'l2']
* Compare the coefficients and the f1score

In [66]:
print ("C = 0.01")
print ("L1", "lasso")

logreg10_1_a = LogisticRegression(C=0.01, penalty='l1', solver='liblinear')
logreg10_1_a.fit(X6_n, Y7)
print(logreg10_1_a.coef_)

Y_10_1_a_predictions = logreg10_1_a.predict(X6_n)
print('Accuracy: \n', accuracy_score(Y7, Y_10_1_a_predictions))
print('f1_Score: \n', f1_score(Y7, Y_10_1_a_predictions)) # Calculate metrics globally by counting the total true positives, false negatives and false positives.

print("L2, ri")
logreg10_1_b = LogisticRegression(C=0.01, penalty='l2', solver='liblinear')
logreg10_1_b.fit(X6_n, Y7)
print(logreg10_1_b.coef_)

Y_10_1_b_predictions = logreg10_1_b.predict(X6_n)
print('Accuracy: \n', accuracy_score(Y7, Y_10_1_b_predictions))
print('f1_Score: \n', f1_score(Y7, Y_10_1_b_predictions)) # Calculate metrics globally by counting the total true positives, false negatives and false positives.

print("C = 0.1")
print("L1")
logreg10_2_a = LogisticRegression(C=0.1, penalty='l1', solver='liblinear')
logreg10_2_a.fit(X6_n, Y7)
print(logreg10_2_a.coef_)

Y_10_2_a_predictions = logreg10_2_a.predict(X6_n)
print('Accuracy: \n', accuracy_score(Y7, Y_10_2_a_predictions))
print('f1_Score: \n', f1_score(Y7, Y_10_2_a_predictions)) # Calculate metrics globally by counting the total true positives, false negatives and false positives.

print("L2")
logreg10_2_b = LogisticRegression(C=0.1, penalty='l2', solver='liblinear')
logreg10_2_b.fit(X6_n, Y7)
print(logreg10_2_b.coef_)

Y_10_2_b_predictions = logreg10_2_b.predict(X6_n)
print('Accuracy: \n', accuracy_score(Y7, Y_10_2_b_predictions))
print('f1_Score: \n', f1_score(Y7, Y_10_2_b_predictions)) # Calculate metrics globally by counting the total true positives, false negatives and false positives.

print ("C = 1")
print("L1")
logreg10_3_a = LogisticRegression(C=1, penalty='l1', solver='liblinear')
logreg10_3_a.fit(X6_n, Y7)
print(logreg10_3_a.coef_)

Y_10_3_a_predictions = logreg10_3_a.predict(X6_n)
print('Accuracy: \n', accuracy_score(Y7, Y_10_3_a_predictions))
print('f1_Score: \n', f1_score(Y7, Y_10_3_a_predictions)) # Calculate metrics globally by counting the total true positives, false negatives and false positives.

print("L2")
logreg10_3_b = LogisticRegression(C=1, penalty='l2', solver='liblinear')
logreg10_3_b.fit(X6_n, Y7)
print(logreg10_3_b.coef_)

Y_10_3_b_predictions = logreg10_3_b.predict(X6_n)
print('Accuracy: \n', accuracy_score(Y7, Y_10_3_b_predictions))
print('f1_Score: \n', f1_score(Y7, Y_10_3_b_predictions)) # Calculate metrics globally by counting the total true positives, false negatives and false positives.


C = 0.01
L1 lasso
[[ 0.         -0.35079754  0.          0.         -0.0242809   0.
   0.          0.          0.          0.07960373  0.84114512  0.        ]]
Accuracy: 
 0.8157611205171618
f1_Score: 
 0.28108108108108104
L2, ri
[[ 0.18429336 -0.37887596  0.02103926  0.31978012 -0.20735008  0.15011426
  -0.18026332 -0.31735527  0.14784796  0.23374954  0.70558531  0.01590001]]
Accuracy: 
 0.8197629675234723
f1_Score: 
 0.3417650365373805
C = 0.1
L1
[[ 0.35521794 -0.55150504 -0.00386492  0.59463244 -0.23841997  0.17567032
  -0.21910094 -0.57972344  0.25703126  0.30527712  0.79849787  0.07731327]]
Accuracy: 
 0.8199168847160228
f1_Score: 
 0.3578485181119649
L2
[[ 0.45774901 -0.5690227  -0.02778742  0.74526984 -0.27643395  0.19688836
  -0.22960076 -0.8108624   0.31236366  0.32510428  0.69935354  0.18108278]]
Accuracy: 
 0.817762044020317
f1_Score: 
 0.35791757049891537
C = 1
L1
[[ 0.61003971 -0.59747018 -0.03274185  0.99727367 -0.26367716  0.19005676
  -0.21068431 -1.19648316  0.40034149

**RTA/** Con un C=0.01 los coeficientes se hacen 0. De igual manera de acuerdo al r1_score el mejor es con C=1 y para penalty = L1. 